<a href="https://colab.research.google.com/github/nrajmalwar/Project/blob/master/Session%2013/Assignment_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [0]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

In [13]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Enable eager execution in tensorflow
tf.enable_eager_execution()

# Hyperparameters

In [0]:
# Changing the batch size to 128
BATCH_SIZE = 128 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.01 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 50 #@param {type:"integer"}

# Model Building

In [0]:
# Function to use values for kernel initializer
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
# Class to perform Convolution2d, BatchNormalization and Relu activation
class ConvBN(tf.keras.Model):
  def __init__(self, c_out, k_size=3, stride=(1,1)):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=k_size, strides=stride, padding="same", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.drop = tf.keras.layers.Dropout(0.05)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.drop(self.conv(inputs))))

In [0]:
# Class to define one Resnet Block
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out, k_size=1)
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)
      self.res3 = ConvBN(c_out)
      self.res4 = ConvBN(c_out)

  # Adding the shortcut connection
  def call(self, inputs):
    h = self.conv_bn(inputs)
    if self.res:
      # Repeating one block twice
      h = h + self.res2(self.res1(h))
      h = h + self.res4(self.res3(h))
    return h

In [0]:
# ResNet18 Model architecture
class ResNet18(tf.keras.Model):
  def __init__(self, c=32, weight=0.125):
    super().__init__()
    # First layer
    # In the defined architecture, it used 7x7 kernel with stride 2
    # We're using 3x3 kernel with stride 1, because the input resolution is just 32x32
    self.init_conv_bn = ConvBN(c, 3, (1,1))

    # Originally used MaxPooling with kernel 3x3 and stride 2
    # We're using MaxPooling with kernel 2x2 and stride 2, gives an output resolution of 16x16
    self.pool1 = tf.keras.layers.MaxPooling2D()

    # First resnet block with 32 kernels, which is used twice. The block is repeated in the class ResBlk
    self.blk1 = ResBlk(c, res = True)
    # Second resnet block with 64 kernels
    self.blk2 = ResBlk(c*2, res = True)
    # Third resnet block with 128 kernels
    self.blk3 = ResBlk(c*4, res = True)
    # Fourth resnet block with 256 kernels
    self.blk4 = ResBlk(c*8, res = True)
    # Average pooling of all the pixels along the channels
    self.pool = tf.keras.layers.GlobalAveragePooling2D()
    # Connect to a dense layer of 10 units
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2((self.blk1(self.pool1(self.init_conv_bn(x))))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [0]:
# Load the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

# We're not using the calculated normalization values
# train_mean = np.mean(x_train, axis=(0,1,2))
# train_std = np.std(x_train, axis=(0,1,2))

# Normalization values provided in the assignment
train_mean = (0.4914, 0.4822, 0.4465)
train_std = (0.2023, 0.1994, 0.2010)

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
#Padding with 4px
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

In [0]:
model = ResNet18()
# Use SGD optimizer with momentum
opt = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE, momentum=MOMENTUM, nesterov=True)
# Do random crop of 32, and horizontal flip
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

# Model Training

In [22]:
test_acc_prev = 0
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

# Model Training
for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var))

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()

  # Save the best model
  if test_acc > test_acc_prev:
    model.save_weights('/content/drive/My Drive/Colab Notebooks/EVA/Session 13/my_model') 
    test_acc_prev = test_acc

  print('epoch:', epoch+1, 'lr:', 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: train loss: 1.6955046745300293 train acc: 0.34928 val loss: 2.1069267730712893 val acc: 0.3394 time: 74.14128971099854



epoch: 2 lr: train loss: 1.279337512664795 train acc: 0.53222 val loss: 1.2382920316696167 val acc: 0.5519 time: 148.75813174247742



epoch: 3 lr: train loss: 1.1015299655151367 train acc: 0.60204 val loss: 1.699995288848877 val acc: 0.454 time: 222.81707072257996



epoch: 4 lr: train loss: 0.9929155990600586 train acc: 0.64402 val loss: 1.4562041244506836 val acc: 0.5547 time: 296.4142792224884



epoch: 5 lr: train loss: 0.916309303741455 train acc: 0.67156 val loss: 1.1219068881988525 val acc: 0.6323 time: 370.6456456184387


...


epoch: 46 lr: train loss: 0.2581636982727051 train acc: 0.90756 val loss: 0.5289528499126435 val acc: 0.8463 time: 3398.3123137950897



epoch: 47 lr: train loss: 0.25018366138458253 train acc: 0.91166 val loss: 0.40377490119934084 val acc: 0.8753 time: 3471.0546083450317



epoch: 48 lr: train loss: 0.25042452993392944 train acc: 0.91202 val loss: 0.4431751935005188 val acc: 0.8678 time: 3544.7276723384857



epoch: 49 lr: train loss: 0.23935777126312255 train acc: 0.91558 val loss: 0.45910451550483705 val acc: 0.8646 time: 3617.98685503006



epoch: 50 lr: train loss: 0.23767643747329711 train acc: 0.91664 val loss: 0.5386530284881592 val acc: 0.8539 time: 3692.19700050354
